In [1]:
import numpy as np
import itertools
import cv2
import random
import copy
from time import sleep, time
import pickle
from numba import jit, prange

### Parâmetros

In [2]:
def initialize_parameters(new):
    params = {}
    params['nlayer'] = 2
    params['wlen'] = 3
    params['wsize'] = params['wlen'] ** 2
    params['dim'] = 56
    params['train_size'] = 30
    params['val_size'] = 10
    params['test_size'] = 10
    params['error_type'] = 'iou'
    params['neighbors_sample'] = 10
    params['epoch_f'] = 100
    params['epoch_w'] = 50
    params['batch'] = 30
    
    params['windows_continuos'] = np.load('windows_continuos.txt', allow_pickle=True)

    if new:
        #Wini = np.array([np.nan, 1., np.nan, 1., 1., 1., np.nan, 1., np.nan])
        params['W'] = [create_window(params['windows_continuos']) for _ in range(params['nlayer'])]
        params['joint'] = [create_joint(params['W'][i]) for i in range(params['nlayer'])]
        params['w_hist'] = {"W":[],"error":[]}
    else:
        params['joint'] = np.load('joint.txt', allow_pickle=True)
        params['W'] = np.load('W.txt', allow_pickle=True)
        params['w_hist'] = np.load('W_hist.txt', allow_pickle=True)

    return params

In [3]:
def create_window(windows_continuos):
    #np.random.seed(0)
    wind =  random.randint(0, len(windows_continuos)-1)
    W = np.array([1. if i=='1' else np.nan for i in windows_continuos[wind]])
    return W

In [4]:
def create_joint(W):
    Ji=[]
    ni = int(W[~np.isnan(W)].sum())
    for i in itertools.product([0, 1], repeat=ni):
        Ji.append(''.join(np.array(i).astype(str)))
    np.random.seed(0)
    return np.c_[Ji, np.random.randint(2, size=len(Ji))]

In [5]:
params = initialize_parameters(new = True)

In [53]:
nlayer = params['nlayer']
wlen = params['wlen']
wsize = params['wsize']
dim = params['dim']
train_size = params['train_size']
val_size = params['val_size']
test_size = params['test_size']
W = params['W']
joint = params['joint']
windows_continuos = params['windows_continuos']
error_type = params['error_type']
w_hist =  params['w_hist']
neighbors_sample = params['neighbors_sample']
epoch_f = params['epoch_f']
epoch_w = params['epoch_w']
batch = params['batch']


In [57]:
w_hist = {"W":[],"error":[]}

In [54]:
W

[array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]),
 array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]

In [55]:
joint

[array([['000', '0'],
        ['001', '1'],
        ['010', '1'],
        ['011', '0'],
        ['100', '1'],
        ['101', '1'],
        ['110', '1'],
        ['111', '1']], dtype='<U11'),
 array([['00', '0'],
        ['01', '1'],
        ['10', '1'],
        ['11', '0']], dtype='<U11')]

In [45]:
increase = int(round(wlen/2-0.1,0))

In [10]:
def convert_binary(img):
    (T, img_bin) = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
    img_bin[(img_bin==0)]=1
    img_bin[(img_bin==255)]=0
    img_bin = img_bin.astype(int)
    return img_bin

In [11]:
train = []
ytrain = []
for img in range(1,train_size+1):
    s = str(img)
    s = s.zfill(2)
    x = cv2.imread('./x/train'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    y = cv2.imread('./y/train'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    train.append(convert_binary(x))
    ytrain.append(convert_binary(y))

In [12]:
val = []
yval = []
for img in range(1,val_size+1):
    s = str(img)
    s = s.zfill(2)
    x = cv2.imread('./x/val'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    y = cv2.imread('./y/val'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    val.append(convert_binary(x))
    yval.append(convert_binary(y))

In [13]:
test = []
ytest = []
for img in range(1,test_size+1):
    s = str(img)
    s = s.zfill(2)
    x = cv2.imread('./x/test'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    y = cv2.imread('./y/test'+s+'.jpg', cv2.IMREAD_GRAYSCALE)
    test.append(convert_binary(x))
    ytest.append(convert_binary(y))

In [14]:
train[0].shape, ytrain[0].shape, val[0].shape, yval[0].shape, test[0].shape, ytest[0].shape

((56, 56), (56, 56), (56, 56), (56, 56), (56, 56), (56, 56))

In [15]:
def save_results(Wtrain, Wval, Wtest):
    for img in range(len(Wtrain)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtrain[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results/train'+s+'.jpg', x)

    for img in range(len(Wval)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wval[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results/val'+s+'.jpg', x)
        
    for img in range(len(Wtest)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtest[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results/test'+s+'.jpg', x)
    

In [16]:
def save_results_complet(Wtrain, Wval, Wtest):
    for img in range(len(Wtrain)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtrain[img][0])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/train_op1_'+s+'.jpg', x)
        x = copy.deepcopy(Wtrain[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/train_op2_'+s+'.jpg', x)

    for img in range(len(Wval)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wval[img][0])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/val_op1_'+s+'.jpg', x)
        x = copy.deepcopy(Wval[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/val_op2_'+s+'.jpg', x)
        
    for img in range(len(Wtest)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtest[img][0])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/test_op1_'+s+'.jpg', x)
        x = copy.deepcopy(Wtest[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/test_op2_'+s+'.jpg', x)
    

In [17]:
def apply_window(x, W_n, j_n):
    Xl = np.c_[np.zeros([x.shape[0], increase], dtype=int), x, np.zeros([x.shape[0], increase], dtype=int)]
    Xl = np.r_[np.zeros([increase, Xl.shape[1]], dtype=int), Xl, np.zeros([increase, Xl.shape[1]], dtype=int)]
    
    z = np.zeros([x.shape[0], x.shape[0]], dtype=int)
    
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            p = Xl[i:i+wlen, j:j+wlen].flatten()
            p = p * W_n
            p = (p[~np.isnan(p)].astype(int))
            p = ''.join(p.astype(str))
            
            indices = np.where(j_n[:, 0] == p)
            if indices[0].size > 0 and j_n[indices[0], 1] == '1':
                z[i, j] = 1
                
    return z

In [18]:
def run_window_hood(sample, sample_size, W_current, joint_current, Wlast, layer):
    Wsample = []
    for k in range(sample_size):
        Wsample_k = [] 
        for i in range(nlayer):
            if layer > i:
                Wsample_k.append(Wlast[k][i])
            elif i==0:
                Wsample_k.append(apply_window(sample[k], W_current[i], joint_current[i]))
            else:
                Wsample_k.append(apply_window(Wsample_k[i-1], W_current[i], joint_current[i]))
        Wsample.append(Wsample_k)
    return Wsample

In [19]:
#Wtrain = run_window_hood(train, train_size, W, joint, W, 0)

In [20]:
#Wval = run_window_hood(val, val_size, W, joint, W, 0)

In [21]:
#Wtest = run_window_hood(test, test_size, W, joint, W, 0)

In [29]:
def calculate_error(y, h, et = 'mae'):
    error = 0
    n_samples = len(y)
    for k in range(n_samples):
        if et == 'mae':
            sample_error = np.abs(h[k][-1] - y[k]).sum()
            error += sample_error / (y[k].size)
        elif et== 'iou':
            tot = np.sum(np.maximum(h[k][-1],y[k])==1)
            interc = np.sum(h[k][-1] +y[k] == 2)
            error += (1 - interc/tot)
    return (error/n_samples)     

In [23]:
def joint_history(joint, nlayer):
    for k in range(nlayer):
        if k==0:
        #joint_temp.append(joint[k][:,1])
            joint_hist = ''.join(joint[k][:,1])
        else:
            joint_hist = joint_hist+''.join(joint[k][:,1])
    return joint_hist


In [24]:
def window_history(W, nlayer, wsize):
    for k in range(nlayer):
        if k==0:
            window_hist = ''.join([''.join(item) for item in np.reshape(W[k], (wsize,)).astype(str)])
        else:
            window_hist = window_hist+''.join([''.join(item) for item in np.reshape(W[k], (wsize,)).astype(str)])
    return window_hist

In [25]:
def sort_neighbot(v, n):
    if n<len(v):
        ix = random.sample(range(len(v)), n)
    else:
        ix = range(len(v))
    return ix

In [64]:
def get_error_window(W, joint):
    #start = time()
    keep = True
    Wtrain = run_window_hood(train, train_size, W, joint, W, 0)
    w_error = calculate_error(ytrain, Wtrain,error_type)
    joint_hist = []
    #print('----------')
   # print('Entrando no get error window')
    #print('start error: ', w_error)
    flg2 = 0
    epoch_min = 0
    #while keep:
    for ep in range(epoch_f):
        #print('Teste ', qq)
        flg = 0
        joint_hist.append(joint_history(joint, nlayer))
        for k in range(nlayer):
            if not neighbors_sample:
                neighbors_to_visit = range(len(joint[k]))
            else:
                neighbors_to_visit = sort_neighbot(joint[k], neighbors_sample)
            for i in neighbors_to_visit:
                #print('testing layer ', k)
                #print('joint ', joint[k][i][0])
                joint_temp = copy.deepcopy(joint)
                if joint[k][i][1] == '1':
                    joint_temp[k][i][1] = '0'
                else:
                    joint_temp[k][i][1] = '1'
                j_temp = joint_history(joint_temp, nlayer)
                if j_temp not in joint_hist:
                    joint_hist.append(j_temp)
                    W_hood = run_window_hood(train, train_size, W, joint_temp, Wtrain, k)
                    error_hood = calculate_error(ytrain, W_hood,error_type)
                    #print('error: ', error_hood)
                    if error_hood < w_error:
                        #print('erro menor')
                        w_error = error_hood
                        #print(w_error)
                        joint_new = copy.deepcopy(joint_temp)
                        Wtrain_new = copy.deepcopy(W_hood)
                        flg = 1
                        flg2 = 1
                    #print('----------')
        if flg == 0:
            #print('Sem erros Menores no teste ', qq)
            keep = False
        else:
            joint = copy.deepcopy(joint_new)
            Wtrain = copy.deepcopy(Wtrain_new)
            #print('error do teste ', qq, ': ', w_error)
            #print('flg = 1 do teste ', qq)
            epoch_min = ep

    #print('----------')
    #print('end of testing, fl2 = ', flg2)

    Wval = run_window_hood(val, val_size, W, joint, W, 0)
    
    error_val = calculate_error(yval, Wval,error_type)
    error = np.array([w_error, error_val])
    print('Função Janela Fixa: ', W, ' / Menor erro na época ', epoch_min, ': ', w_error)
    print('-.-.-.-.-.-.-.-.-.-')
    #end = time()
    #print('tempo de execução: {}'.format(end - start))
    #save_results(Wtrain, Wval,Wtest)

    return (joint, error, flg2, Wtrain, Wval)

In [46]:
joint, error, flg, Wtrain, Wval = get_error_window(W, joint)

----------
Entrando no get error window
start error:  0.9413971332919037
----------
end of testing, fl2 =  1
error da época  4 :  0.8625842749742083
-.-.-.-.-.-.-.-.-.-
tempo de execução: 180.48940515518188


In [58]:
error

array([0.86258427, 0.86251288])

In [48]:
def save_window(joint, W):

    filename_joint = 'joint.txt'
    pickle.dump(joint, open(filename_joint, 'wb'))

    filename_W = 'W.txt'
    pickle.dump(W, open(filename_W, 'wb'))

In [49]:
save_window(joint, W)

In [69]:
def check_great_neighboors(W,joint, error):#(W, nlayer, wsize, wlen, joint, train, train_size, Wtrain, ytrain, error):
    global w_hist
    flg = 0
    print('\n check_great_neighboors Start')
    print('Janela Atual: ', W)
    
    for k in range(nlayer):
        #print('testing layer ', k)
        nan_idx = np.where(np.isnan(W[k]))[0]
        w_line_temp_base = W[k].copy()
        
        for i in nan_idx:
            W_line_temp = w_line_temp_base.copy()
            W_line_temp[i] = 1
            W_line_temp_NN = W_line_temp.copy()
            W_line_temp_NN[np.isnan(W_line_temp_NN)] = 0
            W_line_temp_NN = W_line_temp_NN.astype(int)
                
            if ''.join(W_line_temp_NN.astype(str)) in windows_continuos:
                #print('W in windows continuos')
                W_temp = W.copy()
                W_temp[k] = W_line_temp
                W_h = window_history(W_temp, nlayer, wsize)

                if W_h not in w_hist['W']:
                    print('layer ', k, '/ window ', W_line_temp)
                    joint_temp = joint.copy()
                    joint_temp[k] = create_joint(W_temp[k])

                    joint_temp, w_error, flg2, W_train_temp, W_val_temp = get_error_window(W_temp, joint_temp)
                    #print('new Validation error: ', w_error[1])
                    if w_error[1] < error[1]:
                        #print('erro menor')
                        error_new = w_error
                        joint_new = joint_temp.copy()
                        Wtrain_new = W_train_temp.copy()
                        Wval_new = W_val_temp.copy()
                        W_new = W_temp.copy()
                        save_window(joint_new, W_new)
                        flg = 1
                    #print('----------')

                    w_hist["W"].append(W_h)
                    w_hist["error"].append(w_error)
                    pickle.dump(w_hist, open('W_hist.txt', 'wb'))
            #else:
                #print('W not in window continuos')
    print('flg = ',flg)
    print('-.-.-.-.-.-.-.-.-.-')
    if flg == 1:
        return W_new, joint_new, Wtrain_new, Wval_new, error, flg
    else:
        return W, joint, Wtrain, Wval, error, flg

In [70]:
def check_lesser_neighboors(W,joint, error):#(W, nlayer, wsize, wlen, joint, train, train_size, Wtrain, ytrain, error):
    global w_hist
    flg = 0
    print('\n check_lesser_neighboors Start')
    print('\nJanela Atual: ', W)
    Wval_new = Wval.copy()
    
    for k in range(nlayer):
        #print('testing layer ', k)
        nan_idx = np.where(W[k]==1)[0]
        w_line_temp_base = W[k].copy()
        
        for i in nan_idx:
            W_line_temp = w_line_temp_base.copy()
            W_line_temp[i] = np.nan
            W_line_temp_NN = W_line_temp.copy()
            W_line_temp_NN[np.isnan(W_line_temp_NN)] = 0
            W_line_temp_NN = W_line_temp_NN.astype(int)
            
            if ''.join(W_line_temp_NN.astype(str)) in windows_continuos:
                #print('W in windows continuos')
                W_temp = W.copy()
                W_temp[k] = W_line_temp
                W_h = window_history(W_temp, nlayer, wsize)
                
                if W_h not in w_hist['W']:
                    print('layer ', k, '/ window ', W_line_temp)
                    joint_temp = joint.copy()
                    joint_temp[k] = create_joint(W_temp[k])
                    
                    joint_temp, w_error, flg2, W_train_temp, W_val_temp = get_error_window(W_temp, joint_temp)
                    #print('new Validation error: ', w_error[1])
                    if w_error[1] < error[1]:
                        print('error menor')
                        error[0] = w_error[0]
                        error[1] = w_error[1]
                        joint_new = joint_temp.copy()
                        Wtrain_new = W_train_temp.copy()
                        Wval_new = W_val_temp.copy()
                        W_new = W_temp.copy()
                        print('W_new: ', W_new)
                        save_window(joint_new, W_new)
                        flg = 1
                    #print('----------')
                    
                    w_hist["W"].append(W_h)
                    w_hist["error"].append(w_error)
                    pickle.dump(w_hist, open('W_hist.txt', 'wb'))
                #else:
                    #print('W already in history')
            #else:
                #print('W not in window continuos')
    print('flg = ',flg)
    print('-.-.-.-.-.-.-.-.-.-')
    if flg == 1:
        return W_new, joint_new, Wtrain_new, Wval_new, error, flg
    else:
        return W, joint, Wtrain, Wval, error, flg

In [71]:
w_hist = {"W":[],"error":[]}

In [72]:
start = time()
keep = True
print('start Validation error: ', error[1])
print('----------')

w_hist["W"].append(window_history(W, nlayer, wsize))
w_hist["error"].append(error)
    
pickle.dump(w_hist, open('W_hist.txt', 'wb'))
ep_min=0
#while keep:
for ep in range(epoch_w):
    print('Teste da Época ',ep)
    
    W_l, joint_l, Wtrain_l, Wval_l, error_l, flg_l = check_lesser_neighboors(W,joint, error)
    W_g, joint_g, Wtrain_g, Wval_g, error_g, flg_g = check_great_neighboors(W, joint, error)
    #print(error)
    
    if((error_l[1] < error_g[1])&(flg_l==1)):
        W = W_l.copy()
        joint = joint_l.copy()
        Wtrain = Wtrain_l.copy()
        Wval = Wval_l.copy()
        error = error_l.copy()
        save_window(joint, W)
        #save_results(Wtrain, Wval,Wtest)
        ep_min = ep
        print('época-min: ',ep_min, ' - com erro: ',error )
    elif((error_g[1] < error_l[1])&(flg_g==1)):
        W = W_g.copy()
        joint = joint_g.copy()
        Wtrain = Wtrain_g.copy()
        Wval = Wval_g.copy()
        error = error_g.copy()
        save_window(joint, W)
        #save_results(Wtrain, Wval,Wtest)
        ep_min = ep
        ep_min = ep
        print('época-min: ',ep_min, ' - com erro: ',error )

        
print('--------------') 
Wtest = run_window_hood(test, test_size, W, joint, W, 0)
error_test = calculate_error(ytest, Wtest,error_type)
print('end of testing')#, fl2 = ', flg2)
end = time()
print('tempo de execução: {}'.format(end - start))   
print('época-min: ',ep_min, ' - com erro: ',error )
print('erro de teste: ', error_test)


start Validation error:  0.8625128771231857
----------
Teste da Época  0

 check_lesser_neighboors Start

Janela Atual:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]
layer  0 / window  [nan nan nan  1.  1. nan nan nan nan]
Função Janela Fixa:  [array([nan, nan, nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]  / Menor erro na época  0 :  0.9184266334221245
-.-.-.-.-.-.-.-.-.-
layer  0 / window  [nan  1. nan nan  1. nan nan nan nan]
Função Janela Fixa:  [array([nan,  1., nan, nan,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]  / Menor erro na época  0 :  0.9425247303145274
-.-.-.-.-.-.-.-.-.-
layer  1 / window  [nan  1. nan nan nan nan nan nan nan]
Função Janela Fixa:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([nan,  1., nan, nan, nan, nan, nan, nan, nan])]  / Menor erro na época  2 :  0.8625842749742083
-.-.-.-.-.-.-.-.-.-
layer

flg =  0
-.-.-.-.-.-.-.-.-.-
Teste da Época  17

 check_lesser_neighboors Start

Janela Atual:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]
flg =  0
-.-.-.-.-.-.-.-.-.-

 check_great_neighboors Start
Janela Atual:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]
flg =  0
-.-.-.-.-.-.-.-.-.-
Teste da Época  18

 check_lesser_neighboors Start

Janela Atual:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]
flg =  0
-.-.-.-.-.-.-.-.-.-

 check_great_neighboors Start
Janela Atual:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]
flg =  0
-.-.-.-.-.-.-.-.-.-
Teste da Época  19

 check_lesser_neighboors Start

Janela Atual:  [array([nan,  1., nan,  1.,  1., nan, nan, nan, nan]), array([ 1.,  1., nan, nan, nan, nan, nan, nan, nan])]
flg =  0
-.-.-.-.

end of testing
tempo de execução: 6626.319071769714
época-min:  0  - com erro:  [0.86258427 0.86251288]
erro de teste:  0.9420561749469837


In [ ]:
save_results(Wtrain, Wval, Wtest)

In [ ]:
save_results_complet(Wtrain, Wval, Wtest)

In [ ]:
error

In [ ]:
calculate_error(ytest, Wtest,error_type)

In [ ]:
W

In [62]:
len(w_hist['W'])

13

In [63]:
w_hist['W']

['nan1.0nan1.01.0nannannannan1.01.0nannannannannannannan',
 'nannannan1.01.0nannannannan1.01.0nannannannannannannan',
 'nan1.0nannan1.0nannannannan1.01.0nannannannannannannan',
 'nan1.0nan1.01.0nannannannannan1.0nannannannannannannan',
 'nan1.0nan1.01.0nannannannan1.0nannannannannannannannan',
 '1.01.0nan1.01.0nannannannan1.01.0nannannannannannannan',
 'nan1.01.01.01.0nannannannan1.01.0nannannannannannannan',
 'nan1.0nan1.01.01.0nannannan1.01.0nannannannannannannan',
 'nan1.0nan1.01.0nan1.0nannan1.01.0nannannannannannannan',
 'nan1.0nan1.01.0nannan1.0nan1.01.0nannannannannannannan',
 'nan1.0nan1.01.0nannannannan1.01.01.0nannannannannannan',
 'nan1.0nan1.01.0nannannannan1.01.0nan1.0nannannannannan',
 'nan1.0nan1.01.0nannannannan1.01.0nannan1.0nannannannan']